In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
from typing import List

In [62]:
from selenium_management import start_driver, open_link, get_wait_element, get_wait_elements, close_driver, get_links

In [3]:
driver = start_driver()

### АВТОРИЗАЦИЯ

In [4]:
# Авторизация
def auth(driver: webdriver, username: str, password: str):
    username = 'odintsovmaxim4@gmail.com'
    password = 'password1923DF!'
    open_link(driver=driver, link="https://www.instagram.com/")
    username_element = get_wait_element(driver=driver, by=By.NAME, searched_elem='username')
    username_element.send_keys(username)

    password_element = get_wait_element(driver=driver, by=By.NAME, searched_elem='password')
    password_element.send_keys(password)

    enter_element = get_wait_element(driver=driver, by=By.XPATH, searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div/section/main/article/div[2]/div[1]/div[2]/div/form/div/div[3]/button/div')
    enter_element.click()
    time.sleep(5)
    return True

auth(driver=driver, username='odintsovmaxim4@gmail.com', password='password1923DF!')

True

### Парсинг ссылок на посты

In [5]:
# Переход на страницу с постами
def turn_to_posts_page(driver: webdriver, query: str):
    link = f'https://www.instagram.com/explore/search/keyword/?q={query}'
    open_link(driver=driver, link=link)

In [53]:
def get_post_links(driver: webdriver, wait_time: int = 5, max_scrolls: int = 10) -> set:
    """Скроллинг страниц и поиск ссылок"""

    last_height = driver.execute_script("return document.body.scrollHeight")
    post_links = set()
    max_scrolls += 1
    for i in range(1, max_scrolls):
        # Скролл страницы
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)
        
        # Получение ссылок
        if i % 2 == 0 or i == max_scrolls-1:
            posts_parent_elelemt = get_wait_element(driver=driver, by=By.XPATH, searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[2]')
            new_links = get_links(parent_element=posts_parent_elelemt)
            print(f'Найдены новые ссылки на посты: {new_links}')
            post_links.update(new_links)
            
        # Проверка конца страницы
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    return post_links

In [54]:
turn_to_posts_page(driver=driver, query='%23rapmusic') # '%23r' == '#'
post_links = get_post_links(driver, max_scrolls=1)

Найдены новые ссылки на посты: ['https://www.instagram.com/p/CdeGqjwFzKR/', 'https://www.instagram.com/p/C2LVzZar0P0/', 'https://www.instagram.com/p/C4FuMoJJuPa/', 'https://www.instagram.com/p/C47uxXVoPby/', 'https://www.instagram.com/p/C7meUFOvSte/', 'https://www.instagram.com/p/C_5s_BoIXZV/', 'https://www.instagram.com/p/C8JkUZ3vy-B/', 'https://www.instagram.com/p/C59G_jCvu1U/', 'https://www.instagram.com/p/C17lSNeP2P-/', 'https://www.instagram.com/p/C_8NpjdtPR9/', 'https://www.instagram.com/p/C6wqDbyLMkh/', 'https://www.instagram.com/p/DDKiUVbyw93/', 'https://www.instagram.com/p/C-tCyIjvgIQ/', 'https://www.instagram.com/p/C2pn7LbILCj/', 'https://www.instagram.com/p/C1uK3HVIOec/', 'https://www.instagram.com/p/C69P6qASiuI/', 'https://www.instagram.com/p/C1CgsU4IM6E/', 'https://www.instagram.com/p/C_HOzY0P7uG/']


In [55]:
post_links = list(post_links)
print(f"Найдено {len(post_links)} ссылок на посты")
for href in post_links:
    print(href)

Найдено 18 ссылок на посты
https://www.instagram.com/p/C6wqDbyLMkh/
https://www.instagram.com/p/C69P6qASiuI/
https://www.instagram.com/p/C1uK3HVIOec/
https://www.instagram.com/p/C-tCyIjvgIQ/
https://www.instagram.com/p/C7meUFOvSte/
https://www.instagram.com/p/CdeGqjwFzKR/
https://www.instagram.com/p/C_5s_BoIXZV/
https://www.instagram.com/p/C_8NpjdtPR9/
https://www.instagram.com/p/C1CgsU4IM6E/
https://www.instagram.com/p/C_HOzY0P7uG/
https://www.instagram.com/p/C2pn7LbILCj/
https://www.instagram.com/p/C59G_jCvu1U/
https://www.instagram.com/p/C2LVzZar0P0/
https://www.instagram.com/p/C4FuMoJJuPa/
https://www.instagram.com/p/DDKiUVbyw93/
https://www.instagram.com/p/C17lSNeP2P-/
https://www.instagram.com/p/C8JkUZ3vy-B/
https://www.instagram.com/p/C47uxXVoPby/


### Парсинг ссылок на аккаунты

In [56]:
def accounts_parsing(post_links: List[str]) -> set:
    account_links = set()
    for post_link in post_links:
        find_links = False
        open_link(driver=driver, link=post_link)
        
        # Поиск одного аккаунта в посте
        accounts_parent = get_wait_element(
            driver=driver, 
            by=By.XPATH, 
            searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div[1]/div/div/div/span/span',
            delay=5,
            attempts=1,
            is_error=False
        )
        if accounts_parent:
            print('Найдено в "Поиск одного аккаунта в посте"')
            find_links = True
            
        # Поиск "ещё" в указанных аккаунтах поста
        if find_links is False:
            modal_accounts_element = get_wait_element(
                driver=driver, 
                by=By.XPATH, 
                searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div/div/div/span/span[2]/a/span',
                delay=5,
                attempts=1,
                is_error=False
            )
            if modal_accounts_element and 'ещё' in modal_accounts_element.text:
                modal_accounts_element.click()  # Открытие модального окна с ссылками на аккаунты
                accounts_parent = get_wait_element(
                    driver=driver, 
                    by=By.XPATH, 
                    searched_elem='/html/body/div[6]/div[1]/div/div[2]/div/div/div',
                    sleep=3,
                    delay=5,
                    attempts=1,
                    is_error=False
                )
                if accounts_parent:
                    print('Найдено в "Поиск "ещё" в указанных аккаунтах поста"')
                    find_links = True
        
        # Поиск двух указанных аккаунтов в посте
        if find_links is False:
            accounts_parent = get_wait_element(
                driver=driver, 
                by=By.XPATH, 
                searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div/div/div',
                delay=5,
                attempts=1,
                is_error=False
            )
            if accounts_parent:
                print('Найдено в "Поиск двух указанных аккаунтов в посте"')
                find_links = True
        
        # Поиск ссылок на аккаунты из родительского элемента
        if find_links is True:
            new_links = get_links(accounts_parent)
            print(f'Найдены ссылки: {new_links}')
            account_links.update(new_links)
            
    return account_links


In [57]:
# links = accounts_parsing(['https://www.instagram.com/p/C4TsnmdR6-w/'])
# links
account_links = accounts_parsing(post_links)

Найдено в "Поиск двух указанных аккаунтов в посте"
Найдены ссылки: ['https://www.instagram.com/random_clips98_/', 'https://www.instagram.com/medd_x/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/bunny_therapper/']
Найдено в "Поиск двух указанных аккаунтов в посте"
Найдены ссылки: ['https://www.instagram.com/esco.musicc/', 'https://www.instagram.com/_rap_era_/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/290zipp/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/rapnxt2blow/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/f_u_ck_you_music/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/aabariofficial/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/belkacem_designer_/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instag

In [58]:
account_links = list(account_links)
print(f"Найдено {len(account_links)} ссылок на аккаунты")
for href in account_links:
    print(href)

Найдено 25 ссылок на аккаунты
https://www.instagram.com/ssswaterboi/
https://www.instagram.com/f_u_ck_you_music/
https://www.instagram.com/random_clips98_/
https://www.instagram.com/bunny_therapper/
https://www.instagram.com/aabariofficial/
https://www.instagram.com/upnextartist.__/
https://www.instagram.com/shutter_bug19/
https://www.instagram.com/lamonte_______/
https://www.instagram.com/rapnxt2blow/
https://www.instagram.com/_rap_era_/
https://www.instagram.com/medd_x/
https://www.instagram.com/soumyaa.d_/
https://www.instagram.com/_remon06/
https://www.instagram.com/whatnycsoundslike/
https://www.instagram.com/esco.musicc/
https://www.instagram.com/rapmusic/
https://www.instagram.com/_rik.010/
https://www.instagram.com/world_viral_talent/
https://www.instagram.com/shankha_deep2602/
https://www.instagram.com/music___records__/
https://www.instagram.com/musicsfavoritecolour/
https://www.instagram.com/worldstar__talent__ig__/
https://www.instagram.com/belkacem_designer_/
https://www.i

### Парсинг страничек

In [59]:
def extract_original_link(raw_link):
    """
    Функция извлекает оригинальную ссылку
    """
    from urllib.parse import urlparse, parse_qs
    
    parsed_url = urlparse(raw_link)
    query_params = parse_qs(parsed_url.query)
    orig_link = query_params.get("u", [""])[0]
    if orig_link:
        
        return query_params.get("u", [""])[0]
    return raw_link

# def trim_link(link: str) -> str:
#     """
#     Функция обрезает ссылку, оставляя только основную часть до символа `?`.
#     """
#     if '?' in link:
#         return link.split('?')[0]
#     else:
#         return link




In [ ]:
account_link = account_links[0]
account_link

In [ ]:
link = 'https://www.instagram.com/itsfiftykbaby'
driver.get(link)

In [ ]:
profession_el = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/div[3]/div')
profession_el.text

In [ ]:
# Если одна ссылка
links_el = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/div[4]/div/a/span/span')
links_el.text

In [ ]:
# Ссылка + n ссылок
links_el = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/button/div')
links_el.click()

In [ ]:
links_modal_parent_el = driver.find_element(By.XPATH, "/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div/div")
links = get_links(links_modal_parent_el)
original_links = [trim_link(extract_original_link(link)) for link in links]
print(original_links)

In [ ]:
def parsing_account_info(driver: webdriver, account_link: str) -> dict:
    """
    Парсинг основной информации об аккаунте
    """
    driver.get(account_link)
    time.sleep(3)
    
    links = None
    all_description = None
    
    account_description_parent_element = get_wait_element(  # Элемент со всей указанной информацией об аккаунте
        driver=driver,
        by=By.XPATH,
        searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]',
        delay=3,
        attempts=1,
        is_error=False
    )
    if account_description_parent_element:
        # Парсинг описания аккаунта
        all_description = account_description_parent_element.text

        # Парсинг ссылок, указанных в контактах аккаунта
        account_links_button_element = get_wait_element(  # Кнопка с указанными ссылками
            driver=account_description_parent_element,
            by=By.TAG_NAME,
            searched_elem='button',
            delay=3,
            attempts=1,
            is_error=False
        )
        if account_links_button_element and '+' in account_links_button_element.text:  # Если указано несколько ссылок 
            account_links_button_element.click()  # Открытие модального окна с указанными ссылками
            link_parent_element = get_wait_element(  # Модальное окно с указанными ссылками
                driver=account_links_button_element, 
                by=By.XPATH, 
                searched_elem='/html/body/div[6]/div[1]/div/div[2]/div/div/div/div',
                delay=4,
                attempts=1,
                sleep=2,
                is_error=False
            )
            
            find_links = get_links(link_parent_element)
            links = [extract_original_link(find_link) for find_link in find_links]
        if not links:  # Если в указанных ссылках нет "+ n"
            find_links = get_links(account_description_parent_element)
            links = [extract_original_link(find_link) for find_link in find_links]
        
    print(f'\n___________________\nСсылка на аккаунт: {account_link}\nall_description: {all_description}\nСсылки: {links}')
    account_info = {'account_link': account_link, 'all_description': all_description, 'links': links}
    return account_info


In [69]:
accounts = []
for account_link in account_links:
# for account_link in ['https://www.instagram.com/aabariofficial/']:
    accounts_info = parsing_account_info(driver=driver, account_link=account_link)
    accounts.append(accounts_info)
    
accounts


___________________
Ссылка на аккаунт: https://www.instagram.com/ssswaterboi/
all_description: SSSWATERBÖI
he/him
ssswaterboi
HE’S HIM. 💧
@omegamusicgroupww
youtu.be/0xoyLoEjY_s?si=TxYSJtAt43N2BzKS
Ссылки: ['https://www.threads.net/@ssswaterboi?xmt=AQGzUa8PwN7cP46tcYiEWoyk_wygZC69k-vtdhmynydOhNs', 'https://www.instagram.com/omegamusicgroupww/', 'https://youtu.be/0xoyLoEjY_s?si=TxYSJtAt43N2BzKS&fbclid=PAZXh0bgNhZW0CMTEAAaZWnTMy3ydgHHtd1xfJWGKr0MvsJSurhks2JEbKyWykdUs1UnBK5XUiKeg_aem_Y2QCnxwUYZggj5K2Tl_ZPA']

___________________
Ссылка на аккаунт: https://www.instagram.com/f_u_ck_you_music/
all_description: IloveThisMusic
Музыка
Грехи искупаются не в церкве…это делается на улице.
Ссылки: []

___________________
Ссылка на аккаунт: https://www.instagram.com/random_clips98_/
all_description: Random 💫
random_clips98_
Юморист
🌹استغفرالله العظيم وأتوب إليه 🌹
•Content creator and editor📍AD🇩🇿
•coll📩: benyahyaibrahim79@gmail.com
• facebook :50k... 
ещё
Профиль Facebook
Ссылки: ['https://www.threa

[{'account_link': 'https://www.instagram.com/ssswaterboi/',
  'all_description': 'SSSWATERBÖI\nhe/him\nssswaterboi\nHE’S HIM. 💧\n@omegamusicgroupww\nyoutu.be/0xoyLoEjY_s?si=TxYSJtAt43N2BzKS',
  'links': ['https://www.threads.net/@ssswaterboi?xmt=AQGzUa8PwN7cP46tcYiEWoyk_wygZC69k-vtdhmynydOhNs',
   'https://www.instagram.com/omegamusicgroupww/',
   'https://youtu.be/0xoyLoEjY_s?si=TxYSJtAt43N2BzKS&fbclid=PAZXh0bgNhZW0CMTEAAaZWnTMy3ydgHHtd1xfJWGKr0MvsJSurhks2JEbKyWykdUs1UnBK5XUiKeg_aem_Y2QCnxwUYZggj5K2Tl_ZPA']},
 {'account_link': 'https://www.instagram.com/f_u_ck_you_music/',
  'all_description': 'IloveThisMusic\nМузыка\nГрехи искупаются не в церкве…это делается на улице.',
  'links': []},
 {'account_link': 'https://www.instagram.com/random_clips98_/',
  'all_description': 'Random 💫\nrandom_clips98_\nЮморист\n🌹استغفرالله العظيم وأتوب إليه 🌹\n•Content creator and editor📍AD🇩🇿\n•coll📩: benyahyaibrahim79@gmail.com\n• facebook :50k... \nещё\nПрофиль Facebook',
  'links': ['https://www.threads

### Просмотр и запись данных

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(accounts)
pd.set_option('display.width', None)  # Автоматическая ширина вывода
pd.set_option('display.max_columns', None)  # Показывать все столбцы
df

,account_link,all_description,links
0,https://www.instagram.com/ssswaterboi/,SSSWATERBÖI\nhe/him\nssswaterboi\nHE’S HIM. 💧\...,[https://www.threads.net/@ssswaterboi?xmt=AQGz...
1,https://www.instagram.com/f_u_ck_you_music/,IloveThisMusic\nМузыка\nГрехи искупаются не в ...,[]
2,https://www.instagram.com/random_clips98_/,Random 💫\nrandom_clips98_\nЮморист\n🌹استغفرالل...,[https://www.threads.net/@random_clips98_?xmt=...
3,https://www.instagram.com/bunny_therapper/,Bunny\n👿I LIKE MY Enemies\n🎙Unsigned Artist🎙,[]
4,https://www.instagram.com/aabariofficial/,bari | بَری\nДеятель искусств\n«ریگی»\n@kabodi...,[https://t.me/aabariofficial?fbclid=PAZXh0bgNh...
5,https://www.instagram.com/upnextartist.__/,upnextartist_\nМестные услуги\nMake your music...,[https://www.instagram.com/paco.in.the.thunder/]
6,https://www.instagram.com/shutter_bug19/,"Sibashis Bhattacharya\nФотограф\n📸 Concert, Ev...","[https://www.instagram.com/pixel.mystique_/, h..."
7,https://www.instagram.com/lamonte_______/,Музыкант/группа\n🎨Art | music 🎼\nHere to blend...,[https://linktr.ee/Lamnt247?fbclid=PAZXh0bgNhZ...
8,https://www.instagram.com/rapnxt2blow/,Promoting artists 🔥💪\nPRESS RELEASE || ARTICLE...,[]
9,https://www.instagram.com/_rap_era_/,RAP ERA\nАвтор видео\n2ND PAGE @feelyour2ruth ...,"[https://www.instagram.com/feelyour2ruth/, htt..."


In [ ]:
import json
with open('new_data.json', 'w') as f:
    data = json.dump(accounts, indent=4, ensure_ascii=False, fp=f)

In [ ]:
with open('data.json', 'r') as f:
    loaded_data = json.load(f)
loaded_data

### Запись данных в таблицу

In [78]:
from openpyxl import Workbook

In [80]:
wb = Workbook()
ws = wb.active
ws.title = "Instagram Data"

# Записываем заголовки столбцов
headers = ["Account Link", "Description", "Links"]
ws.append(headers)

# Заполняем данные
for account_item in accounts:
    account_link = account_item['account_link']
    all_description = account_item['all_description']
    links = "\n".join(account_item['links'])  # Преобразуем список ссылок в строку
    ws.append([account_link, all_description, links])

# Сохраняем файл
wb.save("instagram_data.xlsx")
print("Данные успешно записаны в файл 'instagram_data.xlsx'")

Данные успешно записаны в файл 'instagram_data.xlsx'


In [ ]:
from io import BytesIO

from openpyxl import Workbook
from openpyxl.cell.text import InlineFont
from openpyxl.cell.rich_text import CellRichText, TextBlock
from openpyxl.styles import Alignment, PatternFill, Border, Side
from openpyxl.utils.cell import get_column_letter
from openpyxl.styles import PatternFill, Font, Alignment


class PyXLWriter:
    def __init__(self, colors=3):
        self.wb = Workbook()
        self.ws = self.wb.active
        self.colors = colors
        self.widths = {}

    def set_cell(self, pos, value, width=None, height=None):
        row, col = pos
        colname = get_column_letter(col)
        slen = len(value or '') * 1
        if width is None:
            width = max(8, min(12 * 8, slen))
        if height is None:
            height = 15 * (1 + slen // width)
        if height > 15 * 50:
            height = 15 * 50
        old_h = self.ws.row_dimensions[row].height
        self.ws.row_dimensions[row].height = max(height or 25, old_h or 25)
        if col not in self.widths:
            old_w = self.ws.column_dimensions[colname].width or 8
            self.ws.column_dimensions[colname].width = max(width, old_w)
        self.ws.column_dimensions[colname].bestFit = True
        cell = self.ws.cell(row=row, column=col)
        cell.value = value
        if self.colors and row >= self.colors:
            if (row - self.colors) % 2:  # e.g. 2, 4, 6, ...
                cell.fill = PatternFill(start_color='FFFFDD', end_color='FFFFDD', fill_type='solid')
            else:  # e.g. 3, 5, 7, ...
                cell.fill = PatternFill(start_color='DDFFFF', end_color='DDFFFF', fill_type='solid')
        cell.alignment = Alignment(wrap_text=True, vertical='top')
        return cell

    def add_borders(self, style: str = 'dashed', color: str = '000000'):
        """ Добавление линий между всеми заполненными ячейками таблицы """
        border_style = Side(style=style, color=color)
        border = Border(left=border_style, right=border_style, top=border_style, bottom=border_style)

        # Определяем диапазон заполненных ячеек для обработки
        max_row = self.ws.max_row
        max_col = self.ws.max_column

        # Применяем границы ко всем заполненным ячейкам
        for row in range(1, max_row + 1):
            for col in range(1, max_col + 1):
                cell = self.ws.cell(row=row, column=col)
                if cell.value is not None:
                    cell.border = border

    def set_width(self, col, width):
        colname = get_column_letter(col)
        self.ws.column_dimensions[colname].width = width
        self.widths[col] = width

    def __setitem__(self, pos, value):
        self.set_cell(pos, value)

    def save(self, path):
        self.ws.calculate_dimension()
        self.wb.save(path)

In [101]:
###############################
### ЗАПИСЬ ДАННЫХ В ТАБЛИЦУ ###
###############################

pyxl = PyXLWriter(colors=2)
columns_num = 3

# Хедеры
pyxl[1, 1] = "Ссылка на аккаунт"
pyxl[1, 2] = "Описание страницы"
pyxl[1, 3] = "Ссылки из контактов"
# pyxl.set_width(1, 24)


# Запись данных в таблицу
for rid, account_item in enumerate(accounts):
    r = rid + 2
    pyxl[r, 1] = account_item.get('account_link')
    pyxl[r, 2] = account_item.get('all_description')
    pyxl[r, 3] = "\n------------------------------\n".join(account_item.get('links'))

# Стилизация хедеров
for col in range(1, columns_num + 1):
    cell = pyxl.ws.cell(row=1, column=col)
    cell.font = Font(name='Calibri', bold=True, size=13, color='FFFFFF')
    cell.fill = PatternFill(start_color='452bd9', end_color='452bd9', fill_type='solid')
    cell.alignment = Alignment(horizontal='left', vertical='center')

pyxl.add_borders()



# Сохраняем файл
pyxl.save("instagram_data.xlsx")

In [102]:
pyxl.ws.max_column

3